In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_percentage_error
from deap import base, creator, tools, algorithms

In [2]:
data = pd.read_csv('DATA_KOTA.csv')

df = pd.DataFrame(data)

# Normalisasi MinMax
scaler = MinMaxScaler()
df['jumlah_kecelakaan'] = scaler.fit_transform(df[['jumlah_kecelakaan']])

# Menambahkan kolom lag (pergeseran)
df['jumlah_kecelakaan_lag'] = df['jumlah_kecelakaan'].shift(1)
df = df.dropna() # Menghapus baris dengan nilai NaN

In [3]:
# Memisahkan data latih dan data uji sesuai rasio (0.8 untuk 80:20)
train_size = int(0.7 * len(df))
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

# Memisahkan fitur dan target untuk data latih
X_train = train_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_train = train_data['jumlah_kecelakaan'].values

# Memisahkan fitur dan target untuk data uji
X_test = test_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_test = test_data['jumlah_kecelakaan'].values

In [4]:
# Define evaluation function
def evaluate(individual):
    C, epsilon = individual
    if C <= 0 or epsilon <= 0 :
        return float('inf'),
    svr = SVR(kernel='linear', C=C, epsilon=epsilon)
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    return mape,
    
# Setup DEAP framework
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Parameter ranges
C_range = (10, 50)
epsilon_range = (0.001, 1)

def create_individual():
    C = random.uniform(*C_range)
    epsilon = random.uniform(*epsilon_range)
    return creator.Individual([C, epsilon])
    
toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxUniform, indpb=0.5)  # Menggunakan crossover uniform 
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2) # Menggunakan mutasi gaussian
toolbox.register("select", tools.selTournament, tournsize=3) # Menggunakan seleksi turnamen
toolbox.register("evaluate", evaluate)

In [5]:
# Initialize population
population_size = 140
population = toolbox.population(n=population_size)

# Genetic algorithm parameters    
num_generations = 70
crossover_rate = 0.5
mutation_rate = 0.5

# Run genetic algorithm
for generation in range(num_generations):
    print(f"Generasi {generation + 1}:")
    offspring = algorithms.varAnd(population, toolbox, cxpb=crossover_rate, mutpb=mutation_rate)
    for ind in offspring:
        ind[0] = max(min(ind[0], C_range[1]), C_range[0])
        ind[1] = max(min(ind[1], epsilon_range[1]), epsilon_range[0])
        
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
        print(f"Individu: {ind}, MAPE: {fit[0]}")
    
    population = toolbox.select(offspring, k=len(population))

# Get the best individual
best_individual = tools.selBest(population, k=1)[0]
best_mape = best_individual.fitness.values[0]
C, epsilon = best_individual

# Test the best SVR model
best_svr = SVR(kernel='linear', C=C, epsilon=epsilon)
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100

print("\nParameter terbaik:")
print(f"C: {C}")
print(f"Epsilon: {epsilon}")
print(f"MAPE: {mape:.4f}")

Generasi 1:
Individu: [24.76831048531771, 0.6994227642909675], MAPE: 32.4357432690766
Individu: [19.49442331455357, 0.3645852660859156], MAPE: 29.018178305198127
Individu: [21.61267360651562, 0.39520906765831204], MAPE: 32.4357432690766
Individu: [26.895972982672987, 0.013925251971846138], MAPE: 34.56880201704642
Individu: [45.44421785877186, 0.24321230210470512], MAPE: 30.20685989391671
Individu: [13.502262579458622, 0.6773367155506342], MAPE: 32.4357432690766
Individu: [42.903386209523326, 0.6895445575820895], MAPE: 32.4357432690766
Individu: [19.722287555891516, 0.40729119027048916], MAPE: 32.4357432690766
Individu: [34.24727054428886, 0.17952618124611341], MAPE: 36.58883211419345
Individu: [48.91267320209194, 0.7878933707707536], MAPE: 32.4357432690766
Individu: [13.804447373170706, 0.2648562832532251], MAPE: 28.73565057702048
Individu: [35.24294230666687, 0.6648430623058615], MAPE: 32.4357432690766
Individu: [44.778337578416874, 0.11364850955090276], MAPE: 37.0682280645561
Individ

In [6]:
# Prediksi 6 bulan ke depan
last_value = df['jumlah_kecelakaan'].values[-1] # Nilai terakhir dari data sebenarnya
predicts = []

for i in range(6):
    next_value = best_svr.predict(np.array([[last_value]]))[0]
    predicts.append(next_value)
    last_value = next_value

# Denormalisasi prediksi
predicts = scaler.inverse_transform(np.array(predicts).reshape(-1, 1))

print("Prediksi untuk 6 bulan ke depan:")
print(predicts)

Prediksi untuk 6 bulan ke depan:
[[8.35837443]
 [8.74313549]
 [8.85211949]
 [8.88298932]
 [8.89173324]
 [8.89420996]]
